In [ ]:
!pip install beautifulsoup4
!pip install langchain-community
!pip install langchain-openai
!pip install langchain-text-splitter
!pip install openai
!pip install faiss-cpu

ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter


In [ ]:
import getpass
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
import warnings
import openai
import langchain_openai
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document

warnings.filterwarnings('ignore')

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("container-wrapper"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://cesar.breezy.hr/p/00f79174d8ad-pesquisador-em-inteligencia-artificial-e-sistemas-distribuidos",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

4301

### Dividindo Documentos - Splitting/Chunking

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=500, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

6

In [ ]:
print(all_splits[3].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Indexando - Store

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("precisa de doutorado para a vaga?")

len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[0].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Buscando e Recuperando Informações - Retrieve

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


### Gerando Respostas - Generate

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("Tem plano de saúde como benefício?"):
    print(chunk, end="", flush=True)

Sim, tem plano de saúde como benefício no CESAR. Além disso, oferece também plano odontológico e outros auxílios.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [ ]:
texto = """
Era uma vez, no sul dos Estados Unidos, antes da Guerra Civil, um homem chamado Django. Ele era um escravo, acorrentado e sem esperança, até que seu destino cruzou com o de um caçador de recompensas alemão, o Dr. King Schultz. Schultz era um homem peculiar, com modos refinados e um jeito astuto de lidar com os perigos da vida no Velho Oeste.

Certa noite, Schultz encontrou Django e lhe fez uma proposta: se Django o ajudasse a identificar um grupo de criminosos, ele ganharia sua liberdade. Sem outra escolha e movido pela possibilidade de escapar das correntes, Django aceitou.

Assim começou uma parceria improvável. Schultz ensinou Django a manejar uma arma, a montar a cavalo como um homem livre e, principalmente, a enfrentar seu passado sem medo. Durante suas aventuras, Django revelou a Schultz o que mais desejava: encontrar sua esposa, Broomhilda, que fora vendida para longe dele. Movido por um senso de justiça e amizade, Schultz prometeu ajudá-lo.

Os dois seguiram para a imponente fazenda de Calvin Candie, um rico e sádico proprietário de terras conhecido por promover lutas brutais entre escravos. Lá estava Broomhilda, aprisionada e sem saber que Django estava tão perto. Para se infiltrarem, Django e Schultz fingiram ser compradores interessados nos "serviços" de Candie. A tensão cresceu a cada momento, com olhares desconfiados e palavras afiadas trocadas ao redor da mesa de jantar.

Quando a verdade veio à tona, o caos se instaurou. Tiros foram disparados, alianças foram testadas e sacrifícios foram feitos. Schultz, mesmo diante do perigo, manteve sua integridade, mas acabou pagando um preço alto. Django, porém, não desistiu. Com coragem e determinação, ele enfrentou os homens de Candie, salvou Broomhilda e destruiu a fazenda, libertando-se do passado que o aprisionava.

Ao final, Django e Broomhilda partiram juntos, livres para reescrever sua história, enquanto as chamas da fazenda de Candie iluminavam o céu. Django não era mais um escravo. Ele agora era um homem livre, um herói, e sua lenda estava apenas começando.

"""

In [ ]:
Documento = [Document(page_content=texto)]

In [ ]:
# Dividir os documentos em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(Documento)

In [ ]:
# Criar embeddings e indexar no FAISS
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
system_template = """ Você é um conhecedor de bons filmes e irá responder perguntas sobre eles. Diga apenas sobre as histórias e não invente nada

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
# Definição do modelo
llm = ChatOpenAI(model="gpt-4")

# Construir a pipeline RAG
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
question = "Quem ensinou Django a manejar uma arma?"
response = rag_chain.invoke(question)
print("Resposta:", response)

Resposta: Dr. King Schultz ensinou Django a manejar uma arma.
